In [18]:
from sklearn.metrics import confusion_matrix
import datetime as dt
import numpy as np
import pandas as pd

path = ""

In [19]:
#load the labels
test_label = [i.strip() for i in open(path+"test-label.dat").readlines()]
test_label = np.array(test_label)
train_label = [i.strip() for i in open(path+"train-label.dat").readlines()]
train_label = np.array(train_label)

#load the data 3 times (for each classifier,)
dfKNN = pd.read_csv("train_kmedoids.csv") 
df_testKNN = pd.read_csv("test_kmedoids.csv")
dfSVM = pd.read_csv("train_kmedoids.csv") 
df_testSVM = pd.read_csv("test_kmedoids.csv")
dfRF= pd.read_csv("train_kmedoids.csv") 
df_testRF = pd.read_csv("test_kmedoids.csv")

#shape the labels by label (each row represents one label) 
labels_models=[]
for i in range (0,20 ):
    tr=[]
    for j in range (0,len(train_label)):
        tr.append(int(train_label[j][2*i]))
    labels_models.append(tr)
    
train_labels = pd.DataFrame(labels_models)
train_labels = train_labels.transpose()

# test labels
labels__models=[]
for i in range (0,20 ):
    ts=[]
    for j in range (0,len(test_label)):
        ts.append(int(test_label[j][2*i]))
    labels__models.append(ts)
    
test_labels = pd.DataFrame(labels__models)
test_labels = test_labels.transpose()


#normalize the data
from sklearn import preprocessing
x1 = dfKNN.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled1 = min_max_scaler.fit_transform(x1)
dfKNN = pd.DataFrame(x_scaled1)
x1 = df_testKNN.values #returns a numpy array
x_scaled1 = min_max_scaler.transform(x1)
df_testKNN = pd.DataFrame(x_scaled1)
dfinitKNN = dfKNN.copy()



x2 = dfSVM.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled2 = min_max_scaler.fit_transform(x2)
dfSVM = pd.DataFrame(x_scaled2)
x2 = df_testSVM.values #returns a numpy array
x_scaled2 = min_max_scaler.transform(x2)
df_testSVM = pd.DataFrame(x_scaled2)
dfinitSVM = dfSVM.copy()


x3 = dfRF.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled3 = min_max_scaler.fit_transform(x3)
dfRF = pd.DataFrame(x_scaled3)
x3 = df_testRF.values #returns a numpy array
x_scaled3 = min_max_scaler.transform(x3)
df_testRF = pd.DataFrame(x_scaled3)
dfinitRF = dfRF.copy()



In [20]:
def multilabel_confusion_matrix(predict,path,clustnum=-3):
    test_label = [i.strip().split() for i in open(path + "test-label.dat").readlines()]

    for i in range(len(test_label)):
        item = test_label[i]
        for j in range(len(item)):
            item[j] = int(item[j])

        test_label[i] = item

    labels = []

    for item in open(path + "labels.txt").readlines():
        item = item.replace('\n', '')
        label_name, _ = item.split(', ')
        labels.append(label_name)

    y_true = np.array(test_label)
    y_pred = np.array(predict)

    conf_mat_dict = {}

    for label_col in range(len(labels)):
        y_true_label = y_true[:, label_col]
        y_pred_label = y_pred[:, label_col]
        conf_mat_dict[labels[label_col]] = confusion_matrix(y_pred=y_pred_label, y_true=y_true_label)
    f = open("confmatrix"+str(clustnum)+".txt", "w")
    f.close()
    for label, matrix in conf_mat_dict.items():
        print("Confusion matrix for label {}:".format(label))
        f = open("confmatrix"+str(clustnum)+".txt", "a")
        print(matrix)
        f.write("Confusion matrix for label {}:".format(label))
        np.savetxt(f,matrix)
        f.close()
    return

In [21]:
# apply a combination of oversampling and undersampling technique
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
sm = SMOTE(sampling_strategy=0.5, random_state=2)
smsmall = SMOTE(sampling_strategy=0.25, random_state=2)
rm = RandomUnderSampler(random_state=42)


# Fit an ensemble of 10 classifier chains and take the average prediction of all the chains.

orderlist1 = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
orderlist2 = [2,7,4,0,8,6,5,12,10,9,17,11,15,14,3,13,16,1,18,19]
orderlist3 = [1,3,15,5,7,9,11,2,4,13,6,18,8,17,10,14,12,19,16,0]
orderlist4 = [2,4,17,6,8,3,13,19,9,10,14,1,12,0,16,5,7,11,15,18]
orderlist5 = [19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1,0]
orderlist6 = [4,6,3,15,11,7,10,1,13,5,0,12,16,2,18,9,17,8,14,19]
orderlist7 = [15,6,8,0,3,13,18,10,12,1,4,2,17,14,16,11,9,7,19,5]
orderlist8 = [12,7,4,14,16,6,0,19,10,8,3,5,1,9,2,15,18,11,13,17]
orderlist9 = [5,8,19,10,6,12,16,0,3,11,14,4,7,9,17,15,18,13,2,1]
orderlist10 =[19,18,1,16,13,3,14,15,11,17,9,10,12,5,6,8,0,4,7,2]
OrderList = [orderlist1, orderlist2, orderlist3, orderlist4, orderlist5, orderlist6, orderlist7, orderlist8, orderlist9, orderlist10]


# this function ordering the predictions of every chain in the same way
# to fit the confusion matrix correctly
def ordering_labels(order,predictions):
    new_predictions = []
    for pred in predictions:
        temp_predict = [0] * len(pred) 
        for i in range(0,len(pred)):
            temp_predict[ order[i] ] = pred[i]
        new_predictions.append(temp_predict)
    return new_predictions   

In [22]:
###############################################################################
#.................................KNN..........................................
###############################################################################
from sklearn import neighbors
predKNN = pd.DataFrame()
KNNpredictions= []
start_time = dt.datetime.now()
for orderlist in OrderList:
    for l in orderlist:
        #print(train_labels[train_labels[:][l] == 1][l].size)
        if train_labels[train_labels[:][l] == 1][l].size < 1000:
            #print("kkk")
            dfKNN, train_labels_new = smsmall.fit_resample(dfinitKNN,train_labels[:][l])
            dfKNN, train_labels_new = rm.fit_resample(dfKNN,train_labels_new)
        elif train_labels[train_labels[:][l] == 1][l].size < 2000:
            dfKNN, train_labels_new = sm.fit_resample(dfinitKNN,train_labels[:][l])
            dfKNN, train_labels_new = rm.fit_resample(dfKNN,train_labels_new)
        else:
            dfKNN, train_labels_new = rm.fit_resample(dfinitKNN, train_labels[:][l])
        #print(dfKNN.shape)
        KNN = neighbors.KNeighborsClassifier(n_neighbors=3 ,weights='distance',metric='minkowski',p=1).fit(dfKNN, train_labels_new)
        dfinitKNN['Label'+str(l)] = train_labels[:][l]
        df_testKNN['Label'+str(l)] = KNN.predict(df_testKNN) 
        predKNN['Label'+str(l)] = df_testKNN['Label'+str(l)]


    #predKNN = KNN.predict(df_test)
    #print(predKNN)
    predictKNN=[]
    for p in range(3983):
        #print(p)
        resultKNN = []
        for r in range(20):
            x = orderlist.index(r)
            resultKNN.append(predKNN.iloc[p,x])
        predictKNN.append(resultKNN)
    KNNpredictions.append(predictKNN)
finish_time = dt.datetime.now()
time = finish_time-start_time 
print('the training time is:',time)
#multilabel_confusion_matrix(predict,path)

the training time is: 0:02:29.052139


In [23]:
final_predictionsKNN = []
for i in range(0,len(KNNpredictions)):
    final_predictionsKNN.append(ordering_labels( OrderList[i], KNNpredictions[i]))


In [24]:
FinalKNN = []
for i in range(0,len(final_predictionsKNN[0])):
    tempKNN = [0] * len(final_predictionsKNN[0][0])
    for j in range(0,len(final_predictionsKNN)):
        tempKNN = [tempKNN[k] + final_predictionsKNN[j][i][k] for k in range(len(tempKNN))]
    tempKNN = [int(x / len(final_predictionsKNN) + 0.5) for x in tempKNN]
    FinalKNN.append(tempKNN)
    

In [25]:
#...........KNN..................
#confusion matrix for every label
print('for KNN classifier:')
multilabel_confusion_matrix(FinalKNN,path)


for KNN classifier:
Confusion matrix for label programming:
[[2391  615]
 [ 712  265]]
Confusion matrix for label style:
[[3365  390]
 [ 210   18]]
Confusion matrix for label reference:
[[1635  790]
 [1029  529]]
Confusion matrix for label java:
[[3205  406]
 [ 316   56]]
Confusion matrix for label web:
[[2230  703]
 [ 734  316]]
Confusion matrix for label internet:
[[2849  597]
 [ 436  101]]
Confusion matrix for label culture:
[[2439  842]
 [ 480  222]]
Confusion matrix for label design:
[[2512  392]
 [ 929  150]]
Confusion matrix for label education:
[[2792  388]
 [ 689  114]]
Confusion matrix for label language:
[[3046  454]
 [ 435   48]]
Confusion matrix for label books:
[[2993  483]
 [ 444   63]]
Confusion matrix for label writing:
[[2885  620]
 [ 392   86]]
Confusion matrix for label computer:
[[2614  860]
 [ 353  156]]
Confusion matrix for label english:
[[3348  280]
 [ 332   23]]
Confusion matrix for label politics:
[[3130  461]
 [ 316   76]]
Confusion matrix for label history:

In [26]:
#.......KNN.......
#Balanced Accuracy
FinalKNN=pd.DataFrame(FinalKNN)
from sklearn.metrics import balanced_accuracy_score
labels=['programming','style', 'reference', 'java','web', 'internet', 'culture', 'design', 'education', 'language', 'books', 'writing', 'computer','english', 'politics','history','philosophy', 'science','religion', 'grammar']
print('for kNN classifier')
print("the Balanced Accuracy for the labels ")
for i in range (0,20):
    print(labels[i],":",balanced_accuracy_score(test_labels[i], FinalKNN[i]))
    
   


for kNN classifier
the Balanced Accuracy for the labels 
programming : 0.5333238333976877
style : 0.48754292522251036
reference : 0.5068823365933063
java : 0.5190517028315511
web : 0.5306330264802819
internet : 0.5074187977100268
culture : 0.5298051198691247
design : 0.5020156915009051
education : 0.509977521401662
language : 0.48483229813664597
books : 0.4926537678485098
writing : 0.5015130805364721
computer : 0.5294650239273955
english : 0.49380561206267376
politics : 0.5327505271114293
history : 0.507707958018517
philosophy : 0.5389914179843974
science : 0.502571536994936
religion : 0.5020962550151478
grammar : 0.48500341763499655


In [27]:
###############################################################################
#.................................SVM..........................................
###############################################################################

from sklearn import svm
predSVM = pd.DataFrame()

SVMpredictions= []
start_time = dt.datetime.now()

for orderlist in OrderList:
    for l in orderlist:
        #print(train_labels[train_labels[:][l] == 1][l].size)
        if train_labels[train_labels[:][l] == 1][l].size < 1000:
            #print("kkk")
            dfSVM, train_labels_new2 = smsmall.fit_resample(dfinitSVM,train_labels[:][l])
            dfSVM, train_labels_new2 = rm.fit_resample(dfSVM,train_labels_new2)
        elif train_labels[train_labels[:][l] == 1][l].size < 2000:
            dfSVM, train_labels_new2 = sm.fit_resample(dfinitSVM,train_labels[:][l])
            dfSVM, train_labels_new2 = rm.fit_resample(dfSVM,train_labels_new2)
        else:
            dfSVM, train_labels_new2 = rm.fit_resample(dfinitSVM, train_labels[:][l])
        #print(df.shape)
        SVM= svm.SVC(C=1 ,gamma=1.5, kernel='rbf' , random_state=41).fit(dfSVM, train_labels_new2)
        dfinitSVM['Label'+str(l)] = train_labels[:][l]
        df_testSVM['Label'+str(l)] = SVM.predict(df_testSVM) 
        predSVM['Label'+str(l)] = df_testSVM['Label'+str(l)]


    
    predictSVM=[]
    for p in range(3983):
        #print(p)
        resultSVM = []
        for r in range(20):
            x = orderlist.index(r)
            resultSVM.append(predSVM.iloc[p,x])
        predictSVM.append(resultSVM)
    SVMpredictions.append(predictSVM)
finish_time = dt.datetime.now()
time = finish_time-start_time 
print('the training time is:',time)

the training time is: 0:04:01.966437


In [28]:
final_predictionsSVM = []
for i in range(0,len(SVMpredictions)):
    final_predictionsSVM.append(ordering_labels( OrderList[i], SVMpredictions[i]))
    
FinalSVM = []
for i in range(0,len(final_predictionsSVM[0])):
    tempSVM = [0] * len(final_predictionsSVM[0][0])
    for j in range(0,len(final_predictionsSVM)):
        tempSVM = [tempSVM[k] + final_predictionsSVM[j][i][k] for k in range(len(tempSVM))]
    tempSVM = [int(x / len(final_predictionsSVM) + 0.5) for x in tempSVM]
    FinalSVM.append(tempSVM)
    

In [29]:
#...........SVM..................
#confusion matrix for every label
print("for SVM classifier:")
multilabel_confusion_matrix(FinalSVM,path)


for SVM classifier:
Confusion matrix for label programming:
[[2143  863]
 [ 578  399]]
Confusion matrix for label style:
[[3745   10]
 [ 227    1]]
Confusion matrix for label reference:
[[1367 1058]
 [ 773  785]]
Confusion matrix for label java:
[[3558   53]
 [ 369    3]]
Confusion matrix for label web:
[[2579  354]
 [ 872  178]]
Confusion matrix for label internet:
[[3403   43]
 [ 533    4]]
Confusion matrix for label culture:
[[1964 1317]
 [ 342  360]]
Confusion matrix for label design:
[[2898    6]
 [1074    5]]
Confusion matrix for label education:
[[2869  311]
 [ 724   79]]
Confusion matrix for label language:
[[3250  250]
 [ 447   36]]
Confusion matrix for label books:
[[3171  305]
 [ 460   47]]
Confusion matrix for label writing:
[[3412   93]
 [ 469    9]]
Confusion matrix for label computer:
[[2436 1038]
 [ 316  193]]
Confusion matrix for label english:
[[3372  256]
 [ 325   30]]
Confusion matrix for label politics:
[[3328  263]
 [ 365   27]]
Confusion matrix for label history:

In [30]:
#.......SVM.......
#Balanced Accuracy
FinalSVM=pd.DataFrame(FinalSVM)
labels=['programming','style', 'reference', 'java','web', 'internet', 'culture', 'design', 'education', 'language', 'books', 'writing', 'computer','english', 'politics','history','philosophy', 'science','religion', 'grammar']
print("the Balanced Accuracy for the labels ")
for i in range (0,20):
    print(labels[i],":",balanced_accuracy_score(test_labels[i], FinalSVM[i]))
    

the Balanced Accuracy for the labels 
programming : 0.5606502791074283
style : 0.5008614245333707
reference : 0.533781215674338
java : 0.49669357072029013
web : 0.5244141379702238
internet : 0.4974852769680876
culture : 0.555709250619339
design : 0.5012839022970458
education : 0.5002911644227229
language : 0.5015527950310559
books : 0.5024788178390904
writing : 0.49614746417252104
computer : 0.5401919168269932
english : 0.5069723744894948
politics : 0.49781944941719375
history : 0.5018530468841028
philosophy : 0.5306526295560053
science : 0.5412566595700041
religion : 0.5
grammar : 0.4996103896103896


In [31]:
###############################################################################
#..........................Random Forest.....................................
###############################################################################
from sklearn.ensemble import RandomForestClassifier
predRF = pd.DataFrame()
RFpredictions= []
start_time = dt.datetime.now()

      
      
for orderlist in OrderList:
    for l in orderlist:
        #print(train_labels[train_labels[:][l] == 1][l].size)
        if train_labels[train_labels[:][l] == 1][l].size < 1000:
            #print("kkk")
            dfRF, train_labels_new3 = smsmall.fit_resample(dfinitRF,train_labels[:][l])
            dfRF, train_labels_new3 = rm.fit_resample(dfRF,train_labels_new3)
        elif train_labels[train_labels[:][l] == 1][l].size < 2000:
            dfRF, train_labels_new3 = sm.fit_resample(dfinitRF,train_labels[:][l])
            dfRF, train_labels_new3 = rm.fit_resample(dfRF,train_labels_new3)
        else:
            dfRF, train_labels_new3 = rm.fit_resample(dfinitRF, train_labels[:][l])
        #print(df.shape)
        Random_Forest = RandomForestClassifier(n_estimators=20, criterion='gini', random_state=41).fit(dfRF, train_labels_new3)
        dfinitRF['Label'+str(l)] = train_labels[:][l]
        df_testRF['Label'+str(l)] = Random_Forest.predict(df_testRF) 
        predRF['Label'+str(l)] = df_testRF['Label'+str(l)]


    
    predictRF=[]
    for p in range(3983):
        #print(p)
        resultRF = []
        for r in range(20):
            x = orderlist.index(r)
            resultRF.append(predRF.iloc[p,x])
        predictRF.append(resultRF)
    RFpredictions.append(predictRF)
finish_time = dt.datetime.now()
time = finish_time-start_time 
print('the training time is:',time)  

the training time is: 0:00:49.890218


In [32]:
final_predictionsRF = []
for i in range(0,len(RFpredictions)):
    final_predictionsRF.append(ordering_labels( OrderList[i], RFpredictions[i]))
    
FinalRF = []
for i in range(0,len(final_predictionsRF[0])):
    tempRF = [0] * len(final_predictionsRF[0][0])
    for j in range(0,len(final_predictionsRF)):
        tempRF = [tempRF[k] + final_predictionsRF[j][i][k] for k in range(len(tempRF))]
    tempRF = [int(x / len(final_predictionsRF) + 0.5) for x in tempRF]
    FinalRF.append(tempRF)
    

In [33]:
#........Random Forest...........
#confusion matrix for every label
print("for Random Forest classifier:")
multilabel_confusion_matrix(FinalRF,path)

for Random Forest classifier:
Confusion matrix for label programming:
[[2737  269]
 [ 844  133]]
Confusion matrix for label style:
[[3731   24]
 [ 227    1]]
Confusion matrix for label reference:
[[1896  529]
 [1150  408]]
Confusion matrix for label java:
[[3344  267]
 [ 354   18]]
Confusion matrix for label web:
[[2706  227]
 [ 920  130]]
Confusion matrix for label internet:
[[3108  338]
 [ 473   64]]
Confusion matrix for label culture:
[[2501  780]
 [ 438  264]]
Confusion matrix for label design:
[[2742  162]
 [1011   68]]
Confusion matrix for label education:
[[2929  251]
 [ 706   97]]
Confusion matrix for label language:
[[3345  155]
 [ 459   24]]
Confusion matrix for label books:
[[3172  304]
 [ 457   50]]
Confusion matrix for label writing:
[[3090  415]
 [ 403   75]]
Confusion matrix for label computer:
[[3006  468]
 [ 414   95]]
Confusion matrix for label english:
[[3588   40]
 [ 352    3]]
Confusion matrix for label politics:
[[3183  408]
 [ 316   76]]
Confusion matrix for labe

In [34]:
#..Random Forest..
#Balanced Accuracy
FinalRF=pd.DataFrame(FinalRF)
labels=['programming','style', 'reference', 'java','web', 'internet', 'culture', 'design', 'education', 'language', 'books', 'writing', 'computer','english', 'politics','history','philosophy', 'science','religion', 'grammar']
print("the Balanced Accuracy for the labels ")
for i in range (0,20):
    print(labels[i],":",balanced_accuracy_score(test_labels[i], FinalRF[i]))
    

the Balanced Accuracy for the labels 
programming : 0.5233216610109702
style : 0.4989972434414932
reference : 0.521864933896219
java : 0.4872231800680715
web : 0.5232071826343903
internet : 0.5105479486107013
culture : 0.5691679887047153
design : 0.5036180960332111
education : 0.5209330967989536
language : 0.5027018633540373
books : 0.5055812412190211
writing : 0.5192507416183694
computer : 0.5259627228030171
english : 0.4987126729506033
politics : 0.5401300871225683
history : 0.5382126500138923
philosophy : 0.5293018679446604
science : 0.5073669433103924
religion : 0.5281091357569803
grammar : 0.5380587833219412
